In [1]:
import os
import bigfish
import bigfish.stack as stack
import bigfish.multistack as multistack
import bigfish.plot as plot
import pandas as pd
print("Big-FISH version: {0}".format(bigfish.__version__))

Big-FISH version: 0.6.2


## Read in data

In [5]:
path_input = "/data/smFISH/20251028_bartelle_smFISH_mm_microglia_newbuffers/data_r0003_tile0000_1"
path_output = "/home/mauri/big-fish-examples/notebooks"

image_file = os.path.join(path_input, "data_r0003_tile0000_NDTiffStack.tif")
image = stack.read_image(image_file, sanity_check = True)
print("\r shape (z, y, x): {0}".format(image.shape))
print("\r dtype: {0}".format(image.dtype))

metadata = pd.read_csv("/data/smFISH/20251028_bartelle_smFISH_mm_microglia_newbuffers/scan_metadata.csv")
print(metadata.columns)

 shape (z, y, x): (189, 2048, 2048)
 dtype: uint16
Index(['Unnamed: 0', 'root_name', 'scan_type', 'exp_type', 'camera',
       'channels_reversed', 'stage_flipped_x', 'stage_flipped_y',
       'image_rotated', 'image_flipped_y', 'image_flipped_x', 'num_t', 'num_r',
       'num_xyz', 'num_ch', 'na', 'ri', 'z_voxel_um', 'yx_voxel_um', 'binning',
       'overlap', 'blue_active', 'yellow_active', 'red_active',
       'blue_exposure', 'yellow_exposure', 'red_exposure'],
      dtype='object')


In [3]:
# Use this to stack multiple rounds together eventually.
# In my case, the z-slices and the 2 channels are in the same tif file so no need to stack using z or c.

# FOV = tile number
recipe = {
    "fov": "0",
    # "z": str or List[str],
    "c": ["Itgam", "Notch2"],
    "r": "3",
    "ext": "tif",
    # "opt": str,
    "pattern": "data_r000r_tile000fov_NDTiffStack.tif"
}

In [ ]:
# The plot_images function requires a 2D array, so I need a z-slice
image_2d = image[100]
print(image_2d.shape)

In [ ]:
# Rescale pixel intensity
print("smfish")
print("\r min: {0}".format(image_2d.min()))
print("\r max: {0}".format(image_2d.max()), "\n")

image_rescaled = stack.rescale(image_2d, channel_to_stretch=None)
print("smfish rescaled")
print("\r min: {0}".format(image_rescaled.min()))
print("\r max: {0}".format(image_rescaled.max()), "\n")

image_stretched = stack.rescale(image_2d, channel_to_stretch=0)
print("smfish stretched")
print("\r min: {0}".format(image_stretched.min()))
print("\r max: {0}".format(image_stretched.max()))


smfish
 min: 12
 max: 6185 

smfish rescaled
 min: 0
 max: 65535 

smfish stretched
 min: 0
 max: 65535


In [ ]:

images = [image, image_rescaled, image_stretched]
titles = ["original image", "rescaled", "stretched"]
plot.plot_images(image, titles=titles)

(2048, 2048)
